In [2]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import torch
from annoy import AnnoyIndex
import forward_ranker.load_data as load_data
import forward_ranker.train as train
from forward_ranker.utils import print_message
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import pickle
import numpy as np

In [3]:
query_test_emb = obj_reader("/home/jianx/results/test_query_0__emb_p__data_obj_0.pb")
query_test_embid = obj_reader("/home/jianx/results/test_query_0__embid_p__data_obj_0.pb")

In [5]:
len(query_test_emb)
print(query_test_embid)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [16]:
with open("/home/jianx/results/dev_query_0__emb_p__data_obj_0.pb", 'rb') as handle:
    query_dev_emb = pickle.load(handle, encoding="bytes")
with open("/home/jianx/results/dev_query_0__embid_p__data_obj_0.pb", 'rb') as handle:
    query_dev_embid = pickle.load(handle, encoding="bytes")


def generate_annoy_index(embed_size, embeddings, raw_data_path):
    embedding_dict = {}
    with open(raw_data_path, "r") as f:
        rc = 0
        for line in f:
            embedding_dict[int(line.split()[0])] = embeddings[rc]
            rc += 1
    mapping = {}
    i = 0
    index = AnnoyIndex(embed_size, 'dot')
    for key, value in embedding_dict.items():
        index.add_item(i, value)
        mapping[i] = key
        i += 1
        if i % 50000 == 0:
            print_message("Progress: " + str(i) + "/" + str(len(embedding_dict)) + " " + str(i / len(embeddings)))
    return index, mapping

query_dev_index, query_dev_mapping = generate_annoy_index(768, query_dev_emb, "/datadrive/data/raw_data/queries.dev.small.tsv")
print_message("Start Building.")
query_dev_index.build(100)
print_message("Finished Building.")

query_dev_index.save("/datadrive/jianx/data/annoy/" + str(100) + "_ance_query_dev_index.ann")
obj_writer(query_dev_mapping, "/datadrive/jianx/data/annoy/" + str(100) + "_ance_query_dev_map.dict")
print_message("Successfully Saved.")

[Jul 29, 22:18:55] Start Building.
[Jul 29, 22:18:57] Finished Building.
[Jul 29, 22:18:57] Successfully Saved.


In [21]:
query_dev_dict = {}
with open("/datadrive/data/raw_data/queries.dev.small.tsv", "r") as f:
        rc = 0
        for line in f:
            query_dev_dict[rc] = line.split("\t")[1]
            rc += 1
nearest = query_dev_index.get_nns_by_item(2, 1000)
for annoy_id in nearest:
    print(query_dev_dict[annoy_id])

treating tension headaches without medication

average wage gilded age

what are the three monetary policy tools of the fed

what is the best merv rating for filters?

what does human enhancement mean

where was the mlk speech given

green card definition

how do utis happen

what is the best medication for hemorrhoids

what nationality is sanders

what is sunchoke

what sickness has headache and nausea symptoms

is hsa account taxable

what does it mean when you dream about babies

what is pharmaceutical/medical waste

how do you call a rapid response in the hospital?

how much garlic powder replaces a clove of garlic

what is michael strahans salary

who has absolute monarchy

how tall was hulk hogan in his prime

when do you use anova

is glycogen stored in the liver

what does address label mean

where in the genes is the disorder located

when did president obama tickets first vacation while in office

temperature for home freezer

what wages does a police officer and a sheriff ne

In [26]:
passage_embeddings = obj_reader("/home/jianx/results/passage_0__emb_p__data_obj_0.pb")
print(passage_embeddings.shape)

(8841823, 768)


In [ ]:
passage_index, passage_mapping = generate_annoy_index(768, passage_embeddings, "/datadrive/data/raw_data/collection.tsv")
print_message("Start Building.")
passage_index.build(100)
print_message("Finished Building.")

[Aug 01, 19:25:43] Progress: 50000/8841823 0.005654942425334685
[Aug 01, 19:25:49] Progress: 100000/8841823 0.01130988485066937
[Aug 01, 19:25:55] Progress: 150000/8841823 0.016964827276004055
[Aug 01, 19:26:00] Progress: 200000/8841823 0.02261976970133874
[Aug 01, 19:26:06] Progress: 250000/8841823 0.028274712126673426
[Aug 01, 19:26:11] Progress: 300000/8841823 0.03392965455200811
[Aug 01, 19:26:17] Progress: 350000/8841823 0.0395845969773428
[Aug 01, 19:26:22] Progress: 400000/8841823 0.04523953940267748
[Aug 01, 19:26:28] Progress: 450000/8841823 0.050894481828012164
[Aug 01, 19:26:34] Progress: 500000/8841823 0.05654942425334685
[Aug 01, 19:26:39] Progress: 550000/8841823 0.06220436667868153
[Aug 01, 19:26:45] Progress: 600000/8841823 0.06785930910401622
[Aug 01, 19:26:51] Progress: 650000/8841823 0.0735142515293509
[Aug 01, 19:26:56] Progress: 700000/8841823 0.0791691939546856
[Aug 01, 19:27:02] Progress: 750000/8841823 0.08482413638002027
[Aug 01, 19:27:07] Progress: 800000/8841

In [1]:
passage_index.save("/datadrive/jianx/data/annoy/" + str(100) + "_ance_passage_index.ann")
obj_writer(passage_mapping, "/datadrive/jianx/data/annoy/" + str(100) + "_ance_passage_map.dict")
print_message("Successfully Saved.")

NameError: name 'passage_index' is not defined